### **CARTPOLE DQN KERAS**

This is a naive version of the CARTPOLE DQN algorithm <br>
Try to fill the missing parts

This is the simplest approach of DQN with a single network (Remember the standard DQN uses 2 networks and trains one, in this one we use one and train one

Fill the empty places and try to make it to work Ask for help if stocked!!!


In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import random
import gymnasium as gym
import time
import numpy as np
from collections import deque
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf

# Use mixed-precision training for faster computations on supported GPUs
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

import sys
sys.stderr = open('err.txt', 'w')

In [2]:
from tensorflow.python.client import device_lib
devices = device_lib.list_local_devices()
gpu_devices = [device for device in devices if device.device_type == 'GPU']
for gpu in gpu_devices:
    print('Using', gpu.physical_device_desc)

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.19.0
GPUs: []


#### **Hyperparameters**

In [16]:
# Parameters to fine tune
# Try your own parameters
# Remember epsilon and gamma are very important


MAX_EPISODES = 300
ROLLING_WINDOW = 20
MEMORY_SIZE = 2000
MAX_STEPS = 500

gamma = 0.99                        # discount rate
epsilon = 1.0                        # exploration rate
epsilon_min = 0.01
epsilon_decay = 0.99
learning_rate = 0.001
batch_size = 64
solved_threshold = 195

verb = 0                             # to see traces (verbosity)

In [17]:
# Environment setup
env = gym.make("CartPole-v1")
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

#### **Neural Network**

In [6]:
###
# [CREATE YOUR NEURAL NETWORK TRY 16/32 or 24/24]
###

def build_model(state_size, action_size):
    inputs = Input(shape=(state_size,), name="state_input")
    x = Dense(16, activation='relu', name="dense_1")(inputs)
    x = Dense(32, activation='relu', name="dense_2")(x)
    outputs = Dense(action_size, activation='linear', name="output_layer")(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="Q_Network")
    model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))
    return model

#### **Support Functions**

$Q_{\text{target}}(s_t, a_t) = r_t + \gamma \cdot \max_{a'} Q(s_{t+1}, a') \cdot (1 - \text{done})$

In [ ]:
replay_buffer = deque(maxlen=MEMORY_SIZE)

def store(state, action, reward, next_state, done):             
    """
    This function appends the actual observation and reward to the Replay buffer
    """
    state = state.squeeze()
    next_state = next_state.squeeze()
    replay_buffer.append((state, action, reward, next_state, done))

def select_action_greedy(state, model, epsilon):           
    """
    Selects action using epsilon-greedy policy
    """
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    state = np.reshape(state, [1, state_size])
    act_values = model.predict(state, verbose=verb)
    return np.argmax(act_values[0])

# Sample experiences from the replay buffer
def sample_experiences(batch_size):
    """
    Samples a batch_size of experiences from the Replay buffer. 
    You MUST transform the data into numpy arrays as this accelerates the response time sensibily
    """
    indices = np.random.choice(len(replay_buffer), batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    states, actions, rewards, next_states, dones = zip(*batch)

    states = np.array(states)
    actions = np.array(actions)
    rewards = np.array(rewards)
    next_states = np.array(next_states)
    dones = np.array(dones)

    return (states, actions, rewards, next_states, dones)
###
# YOU MUST VECTIORIZE THE RETURN. TRANSFORM THE OUTPUTS IN np arrays otherwise it will be very slow
###

    
def experience_replay(batch_size, model, epsilon):
    """
    The critical function in the whole program
    1. gets a minibatch from replay buffer
    2. Predicts targets_qs from states (we need the full batch predicted but we'll avoid dones)
    3. predicts next_qs from next_states
    4. Bellman equation on next_qs obtains new target_qs $Q_{\text{target}}(s_t, a_t) = r_t + \gamma \cdot \max_{a'} Q(s_{t+1}, a') \cdot (1 - \text{done})$

    6. Train DQN input states, output target_qs 

    """
    if len(replay_buffer) < batch_size:
        return

    states, actions, rewards, next_states, dones = sample_experiences(batch_size)
    
    # Predict Q-values for current and next states using vectorized operations

    
#### 
#  [ Predict target_qs with model predict states, predict next_qs with model predict next states]
####
    target_qs = model.predict(states, verbose=verb) # Get current Q-values for all states
    next_qs = model.predict(next_states, verbose=verb) # Get next Q-values for next states

    # Update target Q-values using standard DQN logic   
    target_qs[np.arange(batch_size), actions] = rewards + gamma * np.max(next_qs, axis=1) * (1 - dones)
    
    # Train the model on the Q-values
    model.fit(states, target_qs, epochs=1, verbose=0)

###
#[Here you have a load and save weights. Can you make a breakpoint to restart the program after a while of training?]
###

def load(name, DQN):
    DQN.load_weights(name)

def save(name, DQN):
    DQN.save_weights(name)

DQN = build_model(state_size, action_size)
DQN.summary()

In [ ]:
# Training loop for DQN
rewards_per_episode = []
rolling_avg_rewards = []
rolling_avg = 0

start_time = time.time()

for episode in range(MAX_EPISODES):
    state, _ = env.reset()
    total_reward = 0
    
    for step in range(MAX_STEPS):
        action = select_action_greedy(state, DQN, epsilon)  # Updated to pass epsilon
        next_state, reward, done, truncated, _ = env.step(action)
        
        store(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
        
        if len(replay_buffer) > batch_size:
            experience_replay(batch_size, DQN, epsilon)
        
        if done or truncated:
            break
    
    # Decay epsilon
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    
    # Track rewards
    rewards_per_episode.append(total_reward)
    rolling_avg = np.mean(rewards_per_episode[-ROLLING_WINDOW:])
    rolling_avg_rewards.append(rolling_avg)
    
    print(f"Episode: {episode+1:3}/{MAX_EPISODES}, Reward: {total_reward:+7.2f}, "
          f"Epsilon: {epsilon:.2f}, Rolling Avg: {rolling_avg:6.2f}, Steps: {step:3}")
    
    # Check if environment is solved
    if rolling_avg >= solved_threshold:
        print(f"Environment solved in {episode+1} episodes!")
        DQN.save("CartPole_dqn_model.keras")
        break

end_time = time.time()
training_duration = (end_time - start_time) / 60  # Convert to minutes
print(f"Training completed in {training_duration:.2f} minutes")

#### **Learning Plot and Episode Rewards**

In [ ]:
# Plot rewards with rolling average

plt.figure(figsize=(10, 6))
plt.plot(rewards_per_episode, label='Rewards', color='blue')
plt.plot(rolling_avg_rewards, label='Rolling Avg (Last 20 Episodes)', color='orange')
plt.axhline(y=solved_threshold, color='red', linestyle='--', label='Solved Threshold')
plt.title('DQN Training Performance CARTPOLE')
plt.xlabel('Episodes')
plt.ylabel('Rewards')
plt.legend()
plt.grid()
plt.show()

#### **Simulation - Testing 10 episodes with the DQN Network**

In [ ]:
# Testing for 10 episodes
start_time = time.time()

for e_test in range(10):  # Run 10 test episodes
    state, _ = env.reset()
    state = np.reshape(state, [1, state_size])
    total_reward = 0
    
    steps = 0
    while True:
        # Use the trained model for testing
        action_vals = DQN.predict(state, verbose=0)  # Predict action values
        action = np.argmax(action_vals[0])  # Choose the action with the highest Q-value

        next_state, reward, done, truncated, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        total_reward += reward
        state = next_state
        steps = steps + 1
        if done or truncated:
            print(f"Test Episode: {e_test + 1:2}/10, Reward: {total_reward:.2f}, Steps: {steps:3}")
            break

end_time = time.time()
testing_duration = (end_time - start_time) / 60  # Convert to minutes
print(f"Testing completed in {testing_duration:.2f} minutes")

#### **Rendering 1 episode**

In [ ]:
# Test the trained agent with video rendering
# This code is useful if you are using colab otherwise use render_mode='human'
env = gym.make(("CartPole-v1"), render_mode='rgb_array')  # Enable RGB rendering
frames = []  # Store frames for visualization

# Render a single test episode
state, _ = env.reset()
state = np.reshape(state, [1, state_size])
tot_rewards = 0

while True:
    # Use the trained model for action
    action_vals = DQN.predict(state, verbose=0)  # Predict action values
    action = np.argmax(action_vals[0])           # Choose the action with the highest Q-value

    next_state, reward, done, truncated, _ = env.step(action)
    frames.append(env.render())                  # Save frame for rendering later
    next_state = np.reshape(next_state, [1, state_size])
    tot_rewards += reward
    state = next_state

    if done or truncated:
        print(f"Rendered Test Episode Reward: {tot_rewards:.2f}")
        break

env.close()

# Save the rendered episode as a GIF
def save_frames_as_gif(frames, path='./', filename='CARTPOLE_DQN.gif'):
    images = [Image.fromarray(frame) for frame in frames]
    gif_path = os.path.join(path, filename)
    images[0].save(gif_path, save_all=True, append_images=images[1:], duration=50, loop=0)
    print(f"Saved GIF to: {gif_path}")

save_frames_as_gif(frames, filename='CARTPOLE_DQN.gif')

## CartPole DDQN

In [ ]:
# DDQN Cartpole

def build_model(state_size, action_size):
    inputs = Input(shape=(state_size,), name="state_input")
    x = Dense(24, activation='relu', name="dense_1")(inputs)
    x = Dense(24, activation='relu', name="dense_2")(x)
    outputs = Dense(action_size, activation='linear', name="output_layer")(x)
    
    model = Model(inputs=inputs, outputs=outputs, name="Q_Network")
    model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))
    return model

# Create main and target networks
DQN = build_model(state_size, action_size)
target_DQN = build_model(state_size, action_size)
target_DQN.set_weights(DQN.get_weights()) # Initialize target network with main network weights

def update_target_network():
    """
    Update the target network with the main network weights
    """
    target_DQN.set_weights(DQN.get_weights())

def select_action_greedy(state, model, epsilon):
    """
    Selects action using epsilon-greedy policy
    """
    if np.random.rand() <= epsilon:
        return random.randrange(action_size)
    state = np.reshape(state, [1, state_size])
    act_values = model.predict(state, verbose=verb)
    return np.argmax(act_values[0])

def experience_replay(batch_size, model, epsilon):
    """
    Sample a batch of experiences from the replay buffer.
    Double DQN: Use the main network to select the action, and the target network to evaluate the Q-values.
    """
    if len(replay_buffer) < batch_size:
        return

    states, actions, rewards, next_states, dones = sample_experiences(batch_size)

    # Get current Q-values from the main network
    target_qs = model.predict(states, verbose=verb)

    # Double DQN: Use the main network to select the action, and the target network to evaluate the Q-values
    next_qs_main = model.predict(next_states, verbose=verb)  # Main network predicts next states
    next_qs_target = target_DQN.predict(next_states, verbose=verb)  # Target network predicts next states

    # Select the best action for each next state using the main network
    best_actions = np.argmax(next_qs_main, axis=1)

    # Get Q-values for the best actions from the target network
    best_qs = next_qs_target[np.arange(batch_size), best_actions]

    # Update the target Q-values using the Bellman equation
    target_qs[np.arange(batch_size), actions] = rewards + gamma * best_qs * (1 - dones)

    # Train the main network on the target Q-values
    model.fit(states, target_qs, epochs=1, verbose=0)

# Training loop for DDQN
rewards_per_episode = []
rolling_avg_rewards = []
rolling_avg = 0

start_time = time.time()

for episode in range(MAX_EPISODES):
    state, _ = env.reset()
    total_reward = 0
    
    for step in range(MAX_STEPS):
        action = select_action_greedy(state, DQN, epsilon)
        next_state, reward, done, truncated, _ = env.step(action)
        
        store(state, action, reward, next_state, done)
        state = next_state
        total_reward += reward
        
        if len(replay_buffer) > batch_size:
            experience_replay(batch_size, DQN, epsilon)
        
        if done or truncated:
            break
    
    # Update target network every 10 episodes
    if episode % 10 == 0:
        update_target_network()
    
    # Decay epsilon
    epsilon = max(epsilon_min, epsilon * epsilon_decay)
    
    # Track rewards
    rewards_per_episode.append(total_reward)
    rolling_avg = np.mean(rewards_per_episode[-ROLLING_WINDOW:])
    rolling_avg_rewards.append(rolling_avg)
    
    print(f"Episode: {episode+1:3}/{MAX_EPISODES}, Reward: {total_reward:+7.2f}, "
          f"Epsilon: {epsilon:.2f}, Rolling Avg: {rolling_avg:6.2f}, Steps: {step:3}")
    
    # Check if environment is solved
    if rolling_avg >= solved_threshold:
        print(f"Environment solved in {episode+1} episodes!")
        DQN.save("CartPole_ddqn_model.keras")
        break

end_time = time.time()
training_duration = (end_time - start_time) / 60  # Convert to minutes
print(f"Training completed in {training_duration:.2f} minutes")

# Plot training results
plt.figure(figsize=(10, 6))
plt.plot(rewards_per_episode, label='Rewards', color='blue')
plt.plot(rolling_avg_rewards, label='Rolling Avg (Last 20 Episodes)', color='orange')
plt.axhline(y=solved_threshold, color='red', linestyle='--', label='Solved Threshold')
plt.title('DDQN Training Performance CARTPOLE')
plt.xlabel('Episodes')
plt.ylabel('Rewards')
plt.legend()
plt.grid()
plt.show()

## Testing Frameworks

In [4]:
import numpy as np
import random
import time
from collections import deque
import gymnasium as gym
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# === Global Settings ===
MAX_EPISODES = 300
ROLLING_WINDOW = 20
MAX_STEPS = 500

gamma = 0.99
epsilon_min = 0.01
batch_size = 64
solved_threshold = 195
verb = 0

# === Replay Buffer (will be resized per experiment) ===
replay_buffer = deque(maxlen=2000)

# === Model Builder ===
def build_model(state_size, action_size, network_structure=(16, 32), learning_rate=0.001):
    inputs = Input(shape=(state_size,))
    x = inputs
    for i, units in enumerate(network_structure):
        x = Dense(units, activation='relu')(x)
    outputs = Dense(action_size, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))
    return model

# === Epsilon-Greedy Action Selection ===
def select_action_greedy(state, model, epsilon):
    if np.random.rand() <= epsilon:
        return random.randrange(model.output_shape[-1])
    q_values = model.predict(state[np.newaxis, :], verbose=verb)
    return np.argmax(q_values[0])

# === Experience Replay Utilities ===
def store(state, action, reward, next_state, done):
    state = state.squeeze()
    next_state = next_state.squeeze()
    replay_buffer.append((state, action, reward, next_state, done))

def sample_experiences(batch_size):
    indices = np.random.choice(len(replay_buffer), batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    states, actions, rewards, next_states, dones = zip(*batch)
    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

def experience_replay(batch_size, model):
    states, actions, rewards, next_states, dones = sample_experiences(batch_size)
    target_qs = model.predict(states, verbose=verb)
    next_qs = model.predict(next_states, verbose=verb)
    target_qs[np.arange(batch_size), actions] = rewards + gamma * np.max(next_qs, axis=1) * (1 - dones)
    model.fit(states, target_qs, epochs=1, verbose=0)

def update_target_network(source_model, target_model):
    target_model.set_weights(source_model.get_weights())

# === Experiment Config Class ===
class ExperimentConfig:
    def __init__(self, network_structure=(16, 32), memory_size=2000,
                 learning_rate=0.001, epsilon_decay=0.99,
                 algorithm="DQN", name="default"):
        self.network_structure = network_structure
        self.memory_size = memory_size
        self.learning_rate = learning_rate
        self.epsilon_decay = epsilon_decay
        self.algorithm = algorithm
        self.name = name

# === Run a Single Experiment ===
def run_experiment(config):
    global replay_buffer
    env = gym.make("CartPole-v1")
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    
    DQN = build_model(state_size, action_size, config.network_structure, config.learning_rate)
    target_DQN = build_model(state_size, action_size, config.network_structure, config.learning_rate) if config.algorithm == "DDQN" else None
    if target_DQN: update_target_network(DQN, target_DQN)
    
    replay_buffer = deque(maxlen=config.memory_size)
    epsilon = 1.0
    rewards_per_episode = []
    rolling_avg_rewards = []
    episodes_to_solve = None
    model_filename = None
    start_time = time.time()

    for episode in range(MAX_EPISODES):
        state, _ = env.reset()
        total_reward = 0

        for step in range(MAX_STEPS):
            action = select_action_greedy(state, DQN, epsilon)
            next_state, reward, done, truncated, _ = env.step(action)
            store(state, action, reward, next_state, done or truncated)
            state = next_state
            total_reward += reward

            if len(replay_buffer) >= batch_size:
                if config.algorithm == "DDQN":
                    states, actions, rewards_, next_states, dones = sample_experiences(batch_size)
                    target_qs = DQN.predict(states, verbose=verb)
                    next_actions = np.argmax(DQN.predict(next_states, verbose=verb), axis=1)
                    next_qs = target_DQN.predict(next_states, verbose=verb)
                    target_qs[np.arange(batch_size), actions] = rewards_ + gamma * next_qs[np.arange(batch_size), next_actions] * (1 - dones)
                    DQN.fit(states, target_qs, epochs=1, verbose=0)
                else:
                    experience_replay(batch_size, DQN)

                if config.algorithm == "DDQN" and episode % 10 == 0:
                    update_target_network(DQN, target_DQN)

            if done or truncated:
                break

        epsilon = max(epsilon_min, epsilon * config.epsilon_decay)
        rewards_per_episode.append(total_reward)
        rolling_avg = np.mean(rewards_per_episode[-ROLLING_WINDOW:])
        rolling_avg_rewards.append(rolling_avg)

        print(f"Config: {config.name}, Ep: {episode+1:3}/{MAX_EPISODES}, "
              f"Reward: {total_reward:7.2f}, Epsilon: {epsilon:.2f}, "
              f"Rolling Avg: {rolling_avg:6.2f}")

        if rolling_avg >= solved_threshold and episodes_to_solve is None:
            episodes_to_solve = episode + 1
            model_filename = f"CartPole_{config.algorithm}_{config.name}_solved.keras"
            DQN.save(model_filename)
            print(f"Model saved as {model_filename}")
            break

    training_duration = (time.time() - start_time) / 60
    if model_filename is None:
        model_filename = f"CartPole_{config.algorithm}_{config.name}_final.keras"
        DQN.save(model_filename)
        print(f"Model saved as {model_filename}")

    return {
        'config': config,
        'episodes_to_solve': episodes_to_solve,
        'training_duration': training_duration,
        'rewards': rewards_per_episode,
        'rolling_avg': rolling_avg_rewards,
        'model_filename': model_filename
    }


def find_best_model(results):
    # Filter only solved models
    solved_models = [r for r in results if r['episodes_to_solve'] is not None]
    
    if not solved_models:
        print("No models reached the solved threshold!")
        return None
    
    # Sort by episodes to solve (ascending) and training duration (ascending)
    best_model = min(solved_models, 
                    key=lambda x: (x['episodes_to_solve'], x['training_duration']))
    
    print("\nBest Model Found:")
    print("----------------")
    print(f"Configuration: {best_model['config'].name}")
    print(f"Algorithm: {best_model['config'].algorithm}")
    print(f"Network Structure: {best_model['config'].network_structure}")
    print(f"Memory Size: {best_model['config'].memory_size}")
    print(f"Learning Rate: {best_model['config'].learning_rate}")
    print(f"Epsilon Decay: {best_model['config'].epsilon_decay}")
    print(f"Episodes to Solve: {best_model['episodes_to_solve']}")
    print(f"Training Duration: {best_model['training_duration']:.2f} minutes")
    print(f"Model File: {best_model['model_filename']}")
    
    return best_model

# Define experiments
experiments = [
    # Network structure comparison (both DQN and DDQN)
    ExperimentConfig(network_structure=(16, 32), algorithm="DQN", name="DQN_16-32"),
    ExperimentConfig(network_structure=(24, 24), algorithm="DQN", name="DQN_24-24"),
    ExperimentConfig(network_structure=(32, 32), algorithm="DQN", name="DQN_32-32"),
    ExperimentConfig(network_structure=(16, 32), algorithm="DDQN", name="DDQN_16-32"),
    ExperimentConfig(network_structure=(24, 24), algorithm="DDQN", name="DDQN_24-24"),
    ExperimentConfig(network_structure=(32, 32), algorithm="DDQN", name="DDQN_32-32"),

    # Replay buffer comparison (DDQN only)
    ExperimentConfig(memory_size=1000, algorithm="DDQN", name="DDQN_mem_1000"),
    ExperimentConfig(memory_size=2000, algorithm="DDQN", name="DDQN_mem_2000"),
    ExperimentConfig(memory_size=5000, algorithm="DDQN", name="DDQN_mem_5000"),

    # Learning rate comparison (DDQN only)
    ExperimentConfig(learning_rate=0.0001, algorithm="DDQN", name="DDQN_lr_0.0001"),
    ExperimentConfig(learning_rate=0.001, algorithm="DDQN", name="DDQN_lr_0.001"),
    ExperimentConfig(learning_rate=0.01, algorithm="DDQN", name="DDQN_lr_0.01"),

    # Epsilon decay comparison (DDQN only)
    ExperimentConfig(epsilon_decay=0.95, algorithm="DDQN", name="DDQN_eps_0.95"),
    ExperimentConfig(epsilon_decay=0.99, algorithm="DDQN", name="DDQN_eps_0.99"),
    ExperimentConfig(epsilon_decay=0.995, algorithm="DDQN", name="DDQN_eps_0.995"),
]

# Run experiments and collect results
results = []
for config in experiments:
    print(f"\nRunning experiment: {config.name}")
    result = run_experiment(config)
    results.append(result)

# Find and print the best model
best_model = find_best_model(results)

# Analyze results
def plot_experiment_results(results):
    plt.figure(figsize=(15, 10))
    
    # Plot rewards for each experiment
    for result in results:
        plt.plot(result['rolling_avg'], label=f"{result['config'].name}")
    
    plt.axhline(y=solved_threshold, color='red', linestyle='--', label='Solved Threshold')
    plt.title('Comparison of Different Configurations')
    plt.xlabel('Episodes')
    plt.ylabel('Rolling Average Reward')
    plt.legend()
    plt.grid()
    plt.show()

# Print summary of results
print("\nExperiment Results Summary:")
print("-------------------------")
for result in results:
    config = result['config']
    print(f"\nConfiguration: {config.name}")
    print(f"Algorithm: {config.algorithm}")
    print(f"Network Structure: {config.network_structure}")
    print(f"Memory Size: {config.memory_size}")
    print(f"Learning Rate: {config.learning_rate}")
    print(f"Epsilon Decay: {config.epsilon_decay}")
    print(f"Episodes to Solve: {result['episodes_to_solve']}")
    print(f"Training Duration: {result['training_duration']:.2f} minutes")
    print(f"Model File: {result['model_filename']}")

# Plot results
plot_experiment_results(results)


Running experiment: DQN_16-32
Config: DQN_16-32, Ep:   1/300, Reward:   10.00, Epsilon: 0.99, Rolling Avg:  10.00
Config: DQN_16-32, Ep:   2/300, Reward:   22.00, Epsilon: 0.98, Rolling Avg:  16.00
Config: DQN_16-32, Ep:   3/300, Reward:   34.00, Epsilon: 0.97, Rolling Avg:  22.00
Config: DQN_16-32, Ep:   4/300, Reward:   23.00, Epsilon: 0.96, Rolling Avg:  22.25
Config: DQN_16-32, Ep:   5/300, Reward:   46.00, Epsilon: 0.95, Rolling Avg:  27.00
Config: DQN_16-32, Ep:   6/300, Reward:   21.00, Epsilon: 0.94, Rolling Avg:  26.00
Config: DQN_16-32, Ep:   7/300, Reward:   20.00, Epsilon: 0.93, Rolling Avg:  25.14
Config: DQN_16-32, Ep:   8/300, Reward:   16.00, Epsilon: 0.92, Rolling Avg:  24.00
Config: DQN_16-32, Ep:   9/300, Reward:   18.00, Epsilon: 0.91, Rolling Avg:  23.33
Config: DQN_16-32, Ep:  10/300, Reward:   26.00, Epsilon: 0.90, Rolling Avg:  23.60
Config: DQN_16-32, Ep:  11/300, Reward:   11.00, Epsilon: 0.90, Rolling Avg:  22.45
Config: DQN_16-32, Ep:  12/300, Reward:   13.

KeyboardInterrupt: 

In [3]:
import os
import numpy as np
import random
import time
from collections import deque
import gymnasium as gym
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam

# === Settings ===
MAX_EPISODES = 300
ROLLING_WINDOW = 20
MAX_STEPS = 500
gamma = 0.99
epsilon_min = 0.01
batch_size = 64
solved_threshold = 195
epsilon_start = 1.0
verb = 0
TARGET_UPDATE_FREQ = 10

# === Model Save Directory ===
MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)

# === Build Model ===
def build_model(state_size, action_size, network_structure=(64, 64), learning_rate=0.001):
    inputs = Input(shape=(state_size,))
    x = inputs
    for units in network_structure:
        x = Dense(units, activation='relu')(x)
    outputs = Dense(action_size, activation='linear')(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))
    return model

# === Replay Logic ===
def select_action_greedy(state, model, epsilon):
    if np.random.rand() <= epsilon:
        return random.randrange(model.output_shape[-1])
    q_values = model.predict(state[np.newaxis, :], verbose=verb)
    return np.argmax(q_values[0])

def store(state, action, reward, next_state, done):
    replay_buffer.append((state.squeeze(), action, reward, next_state.squeeze(), done))

def sample_experiences(batch_size):
    indices = np.random.choice(len(replay_buffer), batch_size, replace=False)
    batch = [replay_buffer[i] for i in indices]
    states, actions, rewards, next_states, dones = zip(*batch)
    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)

def ddqn_experience_replay(main_model, target_model):
    states, actions, rewards, next_states, dones = sample_experiences(batch_size)
    target_qs = main_model.predict(states, verbose=verb)
    next_actions = np.argmax(main_model.predict(next_states, verbose=verb), axis=1)
    next_qs = target_model.predict(next_states, verbose=verb)
    target_qs[np.arange(batch_size), actions] = rewards + gamma * next_qs[np.arange(batch_size), next_actions] * (1 - dones)
    main_model.fit(states, target_qs, epochs=1, verbose=0)

def update_target_network(source_model, target_model):
    target_model.set_weights(source_model.get_weights())

# === Config Class ===
class ExperimentConfig:
    def __init__(self, network_structure=(64, 64), memory_size=2000,
                 learning_rate=0.001, epsilon_decay=0.99, name="default"):
        self.network_structure = network_structure
        self.memory_size = memory_size
        self.learning_rate = learning_rate
        self.epsilon_decay = epsilon_decay
        self.name = name
        self.algorithm = "DDQN"

# === Train One Configuration ===
def run_experiment(config):
    global replay_buffer
    env = gym.make("CartPole-v1")
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n

    main_model = build_model(state_size, action_size, config.network_structure, config.learning_rate)
    target_model = build_model(state_size, action_size, config.network_structure, config.learning_rate)
    update_target_network(main_model, target_model)

    replay_buffer = deque(maxlen=config.memory_size)
    epsilon = epsilon_start
    rewards_per_episode = []
    rolling_avg_rewards = []
    episodes_to_solve = None
    model_filename = None
    start_time = time.time()

    for episode in range(MAX_EPISODES):
        state, _ = env.reset()
        total_reward = 0

        for step in range(MAX_STEPS):
            action = select_action_greedy(state, main_model, epsilon)
            next_state, reward, done, truncated, _ = env.step(action)
            store(state, action, reward, next_state, done or truncated)
            state = next_state
            total_reward += reward

            if done or truncated:
                break

            if len(replay_buffer) >= batch_size:
                ddqn_experience_replay(main_model, target_model)

        if episode % TARGET_UPDATE_FREQ == 0:
            update_target_network(main_model, target_model)

        epsilon = max(epsilon_min, epsilon * config.epsilon_decay)
        rewards_per_episode.append(total_reward)
        rolling_avg = np.mean(rewards_per_episode[-ROLLING_WINDOW:])
        rolling_avg_rewards.append(rolling_avg)

        print(f"Config: {config.name}, Ep: {episode+1:3}/{MAX_EPISODES}, "
              f"Reward: {total_reward:7.2f}, Epsilon: {epsilon:.2f}, "
              f"Rolling Avg: {rolling_avg:6.2f}")

        if rolling_avg >= solved_threshold and episodes_to_solve is None:
            episodes_to_solve = episode + 1
            model_filename = os.path.join(MODEL_DIR, f"CartPole_DDQN_{config.name}_solved.keras")
            main_model.save(model_filename)
            print(f"Model saved as {model_filename}")
            break

    training_duration = (time.time() - start_time) / 60
    if model_filename is None:
        model_filename = os.path.join(MODEL_DIR, f"CartPole_DDQN_{config.name}_final.keras")
        main_model.save(model_filename)
        print(f"Model saved as {model_filename}")

    return {
        'config': config,
        'episodes_to_solve': episodes_to_solve,
        'training_duration': training_duration,
        'rewards': rewards_per_episode,
        'rolling_avg': rolling_avg_rewards,
        'model_filename': model_filename
    }

# === Best Model Finder ===
def find_best_model(results):
    solved_models = [r for r in results if r['episodes_to_solve'] is not None]
    if not solved_models:
        print("No models reached the solved threshold!")
        return None

    best_model = min(solved_models, key=lambda x: (x['episodes_to_solve'], x['training_duration']))
    print("\nBest Model Found:")
    print("----------------")
    print(f"Configuration: {best_model['config'].name}")
    print(f"Algorithm: {best_model['config'].algorithm}")
    print(f"Network Structure: {best_model['config'].network_structure}")
    print(f"Memory Size: {best_model['config'].memory_size}")
    print(f"Learning Rate: {best_model['config'].learning_rate}")
    print(f"Epsilon Decay: {best_model['config'].epsilon_decay}")
    print(f"Episodes to Solve: {best_model['episodes_to_solve']}")
    print(f"Training Duration: {best_model['training_duration']:.2f} minutes")
    print(f"Model File: {best_model['model_filename']}")
    return best_model

# === Define Experiments ===
experiments = [
    # Network structure comparison
    #ExperimentConfig(network_structure=(16, 32), name="DDQN_16-32"),
    ExperimentConfig(network_structure=(24, 24), name="DDQN_24-24"),
    #ExperimentConfig(network_structure=(32, 32), name="DDQN_32-32"),

    # Replay buffer comparison
    ExperimentConfig(memory_size=1000, name="DDQN_mem_1000"),
    #ExperimentConfig(memory_size=2000, name="DDQN_mem_2000"),
    ExperimentConfig(memory_size=5000, name="DDQN_mem_5000"),

    # Learning rate comparison
    ExperimentConfig(learning_rate=0.0001, name="DDQN_lr_0.0001"),
    #ExperimentConfig(learning_rate=0.001, name="DDQN_lr_0.001"),
    ExperimentConfig(learning_rate=0.01, name="DDQN_lr_0.01"),

    # Epsilon decay comparison
    ExperimentConfig(epsilon_decay=0.95, name="DDQN_eps_0.95"),
    #ExperimentConfig(epsilon_decay=0.99, name="DDQN_eps_0.99"),
    ExperimentConfig(epsilon_decay=0.995, name="DDQN_eps_0.995"),
]

# === Run All Experiments ===
results = []
for config in experiments:
    print(f"\nRunning experiment: {config.name}")
    result = run_experiment(config)
    results.append(result)

# === Find Best Model ===
best_model = find_best_model(results)

# === Plot Results ===
def plot_experiment_results(results):
    plt.figure(figsize=(15, 10))
    for result in results:
        plt.plot(result['rolling_avg'], label=f"{result['config'].name}")
    plt.axhline(y=solved_threshold, color='red', linestyle='--', label='Solved Threshold')
    plt.title('Comparison of Different Configurations')
    plt.xlabel('Episodes')
    plt.ylabel('Rolling Average Reward')
    plt.legend()
    plt.grid()
    plt.show()

# === Print Summary ===
print("\nExperiment Results Summary:")
print("-------------------------")
for result in results:
    config = result['config']
    print(f"\nConfiguration: {config.name}")
    print(f"Network Structure: {config.network_structure}")
    print(f"Memory Size: {config.memory_size}")
    print(f"Learning Rate: {config.learning_rate}")
    print(f"Epsilon Decay: {config.epsilon_decay}")
    print(f"Episodes to Solve: {result['episodes_to_solve']}")
    print(f"Training Duration: {result['training_duration']:.2f} minutes")
    print(f"Model File: {result['model_filename']}")

plot_experiment_results(results)


Running experiment: DDQN_24-24
Config: DDQN_24-24, Ep:   1/300, Reward:   16.00, Epsilon: 0.99, Rolling Avg:  16.00
Config: DDQN_24-24, Ep:   2/300, Reward:   34.00, Epsilon: 0.98, Rolling Avg:  25.00
Config: DDQN_24-24, Ep:   3/300, Reward:   25.00, Epsilon: 0.97, Rolling Avg:  25.00
Config: DDQN_24-24, Ep:   4/300, Reward:    9.00, Epsilon: 0.96, Rolling Avg:  21.00
Config: DDQN_24-24, Ep:   5/300, Reward:   30.00, Epsilon: 0.95, Rolling Avg:  22.80
Config: DDQN_24-24, Ep:   6/300, Reward:    9.00, Epsilon: 0.94, Rolling Avg:  20.50
Config: DDQN_24-24, Ep:   7/300, Reward:   15.00, Epsilon: 0.93, Rolling Avg:  19.71
Config: DDQN_24-24, Ep:   8/300, Reward:   17.00, Epsilon: 0.92, Rolling Avg:  19.38
Config: DDQN_24-24, Ep:   9/300, Reward:   60.00, Epsilon: 0.91, Rolling Avg:  23.89
Config: DDQN_24-24, Ep:  10/300, Reward:   20.00, Epsilon: 0.90, Rolling Avg:  23.50
Config: DDQN_24-24, Ep:  11/300, Reward:   28.00, Epsilon: 0.90, Rolling Avg:  23.91
Config: DDQN_24-24, Ep:  12/300, 

KeyboardInterrupt: 